## Importing Libraries

In [133]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import shutil
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Split the Data sets and Validation Sets

In [ ]:
number_of_infected = len(os.listdir(os.path.join('cell_images', 'Parasitized')))
number_of_uninfected = len(os.listdir(os.path.join('cell_images', 'Uninfected')))
print(number_of_infected)
print(number_of_uninfected)

In [ ]:
training_size = number_of_uninfected*.7
validation_size = number_of_uninfected*.2
test_size = number_of_uninfected*.1

In [ ]:
validation_size


In [ ]:
# creating train, validate, and test directories
if not os.path.exists(os.path.join('cell_images', 'train')):
    os.mkdir(os.path.join('cell_images', 'train'))
    os.mkdir(os.path.join('cell_images', 'train', 'infected'))
    os.mkdir(os.path.join('cell_images', 'train', 'uninfected'))

if not os.path.exists(os.path.join('cell_images', 'validate')):
    os.mkdir(os.path.join('cell_images', 'validate'))
    os.mkdir(os.path.join('cell_images', 'validate', 'infected'))
    os.mkdir(os.path.join('cell_images', 'validate', 'uninfected'))

if not os.path.exists(os.path.join('cell_images', 'test')):
    os.mkdir(os.path.join('cell_images', 'test'))
    os.mkdir(os.path.join('cell_images', 'test', 'infected'))
    os.mkdir(os.path.join('cell_images', 'test', 'uninfected'))

In [ ]:
infected_file_lists = os.listdir(os.path.join('cell_images', 'Parasitized'))
uninfected_file_lists = os.listdir(os.path.join('cell_images', 'Uninfected'))

count = 0
for infected_img, uninfected_img in zip(infected_file_lists, uninfected_file_lists):
    if count < training_size:
        src1 = os.path.join('cell_images', 'Parasitized', infected_img)
        dst1 = os.path.join('cell_images', 'train', 'infected', infected_img )
        src2 = os.path.join('cell_images', 'Uninfected', uninfected_img)
        dst2 = os.path.join('cell_images', 'train', 'uninfected', uninfected_img )
        shutil.copy(src1, dst1)
        shutil.copy(src2, dst2)

    if count >= training_size and count < (training_size + validation_size):
        src1 = os.path.join('cell_images', 'Parasitized', infected_img)
        dst1 = os.path.join('cell_images', 'validate', 'infected', infected_img )
        src2 = os.path.join('cell_images', 'Uninfected', uninfected_img)
        dst2 = os.path.join('cell_images', 'validate', 'uninfected', uninfected_img )
        shutil.copy(src1, dst1)
        shutil.copy(src2, dst2)

    if count >= (training_size + validation_size) and count < number_of_infected:
        src1 = os.path.join('cell_images', 'Parasitized', infected_img)
        dst1 = os.path.join('cell_images', 'test', 'infected', infected_img )
        src2 = os.path.join('cell_images', 'Uninfected', uninfected_img)
        dst2 = os.path.join('cell_images', 'test', 'uninfected', uninfected_img )
        shutil.copy(src1, dst1)
        shutil.copy(src2, dst2)
    count +=1

## Data Augumentation and Scaling

In [ ]:
# Define paths
train_dir = os.path.join('cell_images', 'train')
val_dir = os.path.join('cell_images', 'validate')
test_dir = os.path.join('cell_images', 'test')

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Normalization for validation and test data
val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

# Load validation data
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

## Model Building

In [ ]:
model = Sequential()

In [ ]:

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(rate=(.5)))

model.add(Dense(1, activation='sigmoid')) # Binary classification


In [ ]:
model.summary()

In [ ]:
#compiling the model
# we are going to use Binary_Cross_Entropy, because our output will be binary
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

### Train the Model

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

### Model Evaluation

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.2f}')

### Visualizing Accuracy and Loss

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

## Model Testing

In [135]:
# Let's test the model with sample test data 10 for infected and 10 for uninfected
test_img_path = os.listdir('test image')

model_result = {"True Value": [], "Predcited Value": []}

for test_img in test_img_path:
    img_path = os.path.join('test image', test_img)

    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    resized_img = tf.image.resize(img, (64, 64))

    y_pred = model.predict(np.expand_dims(resized_img/255, 0))
    y_pred_label = "uninfected" if y_pred > 0.5 else "infected"
    
    true_y = test_img.split(' ')[0]
    model_result["True Value"].append(true_y)
    model_result["Predcited Value"].append(y_pred_label)

pd.DataFrame(model_result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


,True Value,Predcited Value
0,infected,infected
1,infected,infected
2,infected,infected
3,infected,infected
4,infected,infected
5,infected,infected
6,infected,uninfected
7,infected,infected
8,infected,infected
9,infected,infected


## Model Saving

In [138]:
model.save(os.path.join('Model', 'MalariaImageClassifier.h5'))